In [1]:
import nest_asyncio
nest_asyncio.apply()

import telebot
import librosa
import numpy as np
import tensorflow as tf
import io

In [2]:
TOKEN = os.getenv('TOKEN')
bot = telebot.TeleBot(TOKEN)

In [3]:
model = tf.keras.models.load_model('number_recognizer.h5')


In [4]:
def process_voice(voice_data):
    """Convert Telegram voice message to model input format"""
    # Convert bytes to audio
    y, sr = librosa.load(io.BytesIO(voice_data), sr=16000)
    
    # Extract features
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max).T
    
    # Pad/trim to 10 time steps (adjust if your model expects different)
    if log_mel_spec.shape[0] < 10:
        log_mel_spec = np.pad(log_mel_spec, ((0, 10 - log_mel_spec.shape[0]), (0, 0)))
    else:
        log_mel_spec = log_mel_spec[:10, :]
    
    return log_mel_spec[np.newaxis, ..., np.newaxis] 

In [5]:
@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message, "🎤 Send me a voice message saying a number (0-9) and I'll recognize it!")


In [6]:
@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    try:
        # Get voice file
        file_info = bot.get_file(message.voice.file_id)
        voice_data = bot.download_file(file_info.file_path)
        
        # Process and predict
        features = process_voice(voice_data)
        pred = model.predict(features)
        number = np.argmax(pred)
        
        bot.reply_to(message, f"🔢 I think you said: {number}")
    
    except Exception as e:
        bot.reply_to(message, f"❌ Error: {str(e)}")


In [7]:
bot.polling(none_stop=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)